In [28]:
import csv
import requests

In [29]:
# Book schema
# ISBN-13 no dashes
# Title
# Author(s)
# Publisher
# Publish year
# Dewey decimal
# Categories

cols = [
    "isbn",
    "title",
    "author",
    "publisher",
    "year_published",
    "dewey_decimal",
    "lcc_number",
    "subjects"
]

In [30]:
def to_f(n_str):
    try:
        return float(n_str)
    except ValueError:
        return 0
    
def choose_ddc(book, record):
    if not record.get('dewey_normal'):
        return book['dewey_decimal']
    if str(record['dewey_normal']) == "0":
        return book['dewey_decimal']
    book_d = to_f(book['dewey_decimal'])
    record_d = to_f(record['dewey_normal'])
    if book_d > record_d:
        return str(book_d)
    
    return str(record_d)

In [32]:
books = []
n = 0
with open("books.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for book in reader:
        n += 1
        r = requests.get("http://isbndb.com/api/v2/json/UA15OA8O/book/" + book['isbn'])
        if r.status_code == 200:
            try:
                record = r.json()['data'][0]
            except KeyError:
                print(r.text)
                record = {}
                continue
            if record.get("title"):
                book["title"] = record["title"]
            if record.get('publisher_name'):
                book['publisher'] = record['publisher_name']
            if record.get('author_data'):
                book['author'] = "|".join(author['name'] for author in record['author_data'])
            book['dewey_decimal'] = choose_ddc(book, record)
            book['lcc_number'] = record['lcc_number']
            book['subjects'] = "|".join(record['subject_ids'])
        else:
            print(r.status_code, r.text)
        books.append(book)

{
   "error" : "Unable to locate 1482203537"
}

{
   "error" : "Unable to locate 0813816173"
}

{
   "error" : "Unable to locate 1599943409"
}

{
   "error" : "Unable to locate 1439849633"
}

{
   "error" : "Unable to locate 325721538X"
}

{
   "error" : "Unable to locate 037303444X"
}



In [33]:
len(books)

402

In [34]:
with open("verified.csv", "w", encoding="UTF-8") as csvout:
    writer = csv.DictWriter(csvout, fieldnames=cols, delimiter="\t")
    writer.writeheader()
    for book in books:
        writer.writerow(book)